In [1]:
import multiprocessing as mp
import pandas as pd
import numpy as np
import glob
from datetime import date, datetime, time, timedelta
from collections import defaultdict
import os

utilizando multiprocess, obtendre las temperaturas minimas de cada dia.
Primero, para evitar problemas, desplazare todos los dias en -20 hrs, de forma que cada dia empiece a las 20 hrs y termine a las 20 hrs del dia siguiente, asi obtendre la temperatura minima leida durante la noche (el dia es irrelevante)

In [5]:
path = "../data/cleared"
dfs = []
dict_meta = {"region": [], "estacion": [], "codigo": []}
for file in glob.glob(path + "/*/*"):
    print(file + "                                                                               ", end="\r")
    file = file.replace("\\", "/")
    dict_meta["region"].append(file.split("/")[-2])
    dict_meta["estacion"].append(file.split("/")[-1].split(".")[0])
    df = pd.read_csv(file)
    dict_meta["codigo"].append(df["Codigo"][0])
    dfs.append(df)
    

In [6]:
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['Codigo', 'Fecha', 'Hora'])
df = df.reset_index(drop=True)

In [8]:
df.head()

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica
0,13,2010-06-11,12:15:00,6.2,70.2,179.9,0.0,0.0,125.5,1008.0
1,13,2010-06-11,12:30:00,6.4,64.8,138.4,0.0,0.0,181.7,1007.0
2,13,2010-06-11,12:45:00,6.4,64.9,96.9,0.0,0.0,159.5,1007.0
3,13,2010-06-11,13:00:00,6.9,62.1,121.0,0.0,0.0,328.2,1008.0
4,13,2010-06-11,13:15:00,7.1,62.0,47.0,0.0,0.0,195.2,1007.0


In [11]:
# calculamos la nueva columna datetime y displaced_date
temp = np.array(df["Temperatura"])
code = np.array(df["Codigo"])
ddate = np.array(df["Fecha"])
dtime = np.array(df["Hora"])

ddtt = []
displaced_date = []
# create datetime object and displaceit by -19 hours
deltatime = timedelta(hours = 20)
c = 0
print(len(ddate))
for d, t in zip(ddate, dtime):
    
    if c % 10000 == 0:
        print(c, end="\r")
    if " " in d:
        d = d.split(" ")[0]
        datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
    elif "-" in d:
        datm = datetime.strptime(d + " " + t, '%Y-%m-%d %H:%M:%S')
    else:
        datm = datetime.strptime(d + " " + t, '%d/%m/%Y %H:%M:%S')
    displaced_date.append((datm - deltatime).date().__str__())
    ddtt.append(datm)
    c += 1
    
    
# y la agregamos

df["datetime"] = ddtt
df["displaced_date"] = displaced_date

5301924


In [12]:
# botamos cualquier duplicado que exista, por ABC motivo
print(df.count())
df = df.drop_duplicates(subset=['Codigo', 'Fecha', 'Hora'], keep="first")
print(df.count())

Codigo                 5301924
Fecha                  5301924
Hora                   5301924
Temperatura            5301924
Humedad                5301924
Direccion de Viento    5301924
Velocidad de Viento    5301924
Precipitacion          5301924
Radiacion Solar        5301924
Presion Atmosferica    5301924
datetime               5301924
displaced_date         5301924
dtype: int64
Codigo                 5257742
Fecha                  5257742
Hora                   5257742
Temperatura            5257742
Humedad                5257742
Direccion de Viento    5257742
Velocidad de Viento    5257742
Precipitacion          5257742
Radiacion Solar        5257742
Presion Atmosferica    5257742
datetime               5257742
displaced_date         5257742
dtype: int64


In [15]:
# save this new dataframe
outdir = "../data/processed"
if not os.path.exists(outdir):
    os.makedirs(outdir)
df.to_csv(outdir + "/backup_dffull.csv", index=False, header=True)

## por precaucion, es recomendable resetear el notebook y cargar directamente el df creado, para limpiar memoria

In [2]:
outdir = "../data/processed"
df = pd.read_csv(outdir + "/backup_dffull.csv")
df = df.sort_values(by=['Codigo', 'displaced_date', 'Hora'])
df = df.reset_index(drop=True)
df

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica,datetime,displaced_date
0,13,2010-06-11,12:15:00,6.2,70.2,179.9,0.0,0.0,125.5,1008.0,2010-06-11 12:15:00,2010-06-10
1,13,2010-06-11,12:30:00,6.4,64.8,138.4,0.0,0.0,181.7,1007.0,2010-06-11 12:30:00,2010-06-10
2,13,2010-06-11,12:45:00,6.4,64.9,96.9,0.0,0.0,159.5,1007.0,2010-06-11 12:45:00,2010-06-10
3,13,2010-06-11,13:00:00,6.9,62.1,121.0,0.0,0.0,328.2,1008.0,2010-06-11 13:00:00,2010-06-10
4,13,2010-06-11,13:15:00,7.1,62.0,47.0,0.0,0.0,195.2,1007.0,2010-06-11 13:15:00,2010-06-10
...,...,...,...,...,...,...,...,...,...,...,...,...
5257737,450050,2010-06-29,22:45:00,5.6,91.2,95.0,0.7,0.2,1.0,1017.0,2010-06-29 22:45:00,2010-06-29
5257738,450050,2010-06-29,23:00:00,5.5,91.5,14.0,0.4,0.6,1.0,1017.0,2010-06-29 23:00:00,2010-06-29
5257739,450050,2010-06-29,23:15:00,5.2,93.0,122.0,0.9,0.2,1.0,1017.1,2010-06-29 23:15:00,2010-06-29
5257740,450050,2010-06-29,23:30:00,5.0,94.5,135.0,0.9,0.4,1.0,1016.8,2010-06-29 23:30:00,2010-06-29


In [24]:
df["datetime"] = pd.to_datetime(df["datetime"])

In [25]:
# trabajaremos con la matrix para calcular las temperaturas minimas, es mas eficiente
data = df.values
print(df.columns)
type(data)

Index(['Codigo', 'Fecha', 'Hora', 'Temperatura', 'Humedad',
       'Direccion de Viento', 'Velocidad de Viento', 'Precipitacion',
       'Radiacion Solar', 'Presion Atmosferica', 'datetime', 'displaced_date'],
      dtype='object')


numpy.ndarray

agrupamos los datos en N subset, como diferentes estaciones pueden medir el mismo dia, separaremos por estaciones

In [4]:
# agrupamos los datos en N subset, tienen que ser dias completos, por lo que veamos primero cuantos dias son
total_codes = np.unique(df["Codigo"])
len(total_codes)

162

In [5]:
# uno puede elegir el tamaño de N, sera directamente la cantidad de workers que se lanzaran
N = 24
codes_per_worker = len(total_codes) // N
print(codes_per_worker)

6


In [6]:
ranges = [0]
c = 0
last_code = data[0][0]
i = 0
while i < df.shape[0]:
    if i % 1000 == 0:
        print(i, end="\r")
    di = data[i]
    if last_code != di[0]:
        c += 1
        last_code = di[0]
        if c == codes_per_worker:
            ranges.append(i)
            c = 0
    i += 1
ranges.append(data.shape[0])
ranges = np.array(ranges)
print(ranges.shape)
ranges

(28,)00


array([      0,   26678,   77475,  161990,  242822,  366412,  526204,
        749696, 1010546, 1286809, 1560976, 1836371, 2080444, 2352802,
       2641894, 2811515, 2987416, 3229789, 3517675, 3782765, 4015450,
       4205623, 4439147, 4610683, 4707174, 4913502, 5051518, 5257742])

In [7]:
# verifiquemos que este correcto
for i in range(len(ranges) - 1):
    print(len(np.unique(df.iloc[ranges[i]:ranges[i+1]]["Codigo"])), end=" ")

6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 

In [8]:
# ahora definimos el worker

def worker(data, ini, end, out_q):
    if True:
        results = defaultdict(list)
        print("processing range [{},{}]".format(ini, end))
        # primero obtenemos los sub indices de cada estacion (codigo)
        print("obteneindo sub indices ...", end="")
        sub_idxs = [ini]
        last_code = data[ini][0]
        for i in range(ini, end):
            if last_code != data[i][0]:
                last_code = data[i][0]
                sub_idxs.append(i)
        sub_idxs.append(end)
        print("done")
        
        # luego, recorremos estos rangos, identificamos el rango donde se encuentre las mediciones
        # para 1 solo dia y encontramos el idx donde ocurre la temperatura minima
        
        print("recorriendo datos ...")
        for i in range(len(sub_idxs) - 1):
            subini = sub_idxs[i]
            subend = sub_idxs[i+1]
            last_day = data[subini][11]
            subsubini = subini
            for j in range(subini, subend):
                dj = data[j]
                if last_day != data[j][11]:
                    last_day = data[j][11]
                    # obtenemos el arreglo de temperaturas
                    temps = data[subsubini:j,3]
                    idx_min = np.argmin(temps)
                    # y pasamos todo al diccionaro de salida
                    for k, name in zip([0, 1, 2, 10, 11, 3], 
                                   ["Codigo", "Fecha", "Hora", "datetime", "displaced_date", "minT"]):
                        results[name].append(data[subsubini + idx_min][k])
                    subsubini = j
        print("done")
                    
        out_q.put(results)
#     except:
#         print("worker failed")
#     finally:
#         print("done")


In [9]:
# probemos el worker en un pequeño rango de 1 codigo
idxs = df[df["Codigo"] == 450050].index
ini = idxs[0]
end = idxs[-1]
print(ini, end, end-ini)
m = mp.Manager()
result_queue = m.Queue()
worker(data, ini, end, result_queue)
x = result_queue.get()
pd.DataFrame(x).head()

5225570 5257741 32171
processing range [5225570,5257741]
obteneindo sub indices ...done
recorriendo datos ...
done


,Codigo,Fecha,Hora,datetime,displaced_date,minT
0,450050,2009-07-01,19:45:00,2009-07-01 19:45:00,2009-06-30,2.3
1,450050,2009-07-02,08:15:00,2009-07-02 08:15:00,2009-07-01,1.5
2,450050,2009-07-02,22:30:00,2009-07-02 22:30:00,2009-07-02,4.1
3,450050,2009-07-04,04:45:00,2009-07-04 04:45:00,2009-07-03,2.6
4,450050,2009-07-05,05:45:00,2009-07-05 05:45:00,2009-07-04,5.1


In [10]:
# otra prueba
ini = ranges[1]
end = ranges[2]
print(ini, end, end-ini)
m = mp.Manager()
result_queue = m.Queue()
worker(data, ini, end, result_queue)
x = result_queue.get()
pd.DataFrame(x).head()

26678 77475 50797
processing range [26678,77475]
obteneindo sub indices ...done
recorriendo datos ...
done


,Codigo,Fecha,Hora,datetime,displaced_date,minT
0,38,2010-04-29,07:15:00,2010-04-29 07:15:00,2010-04-28,14.5
1,38,2010-04-29,21:45:00,2010-04-29 21:45:00,2010-04-29,15.1
2,38,2010-05-01,03:15:00,2010-05-01 03:15:00,2010-04-30,17.6
3,38,2010-05-02,04:15:00,2010-05-02 04:15:00,2010-05-01,14.5
4,38,2010-05-03,07:00:00,2010-05-03 07:00:00,2010-05-02,12.6


In [14]:
# all data (un poco lento)
m = mp.Manager()
result_queue = m.Queue()
ini = 0
end = data.shape[0]
worker(data, ini, end, result_queue)
x = result_queue.get()
df_out = pd.DataFrame(x)
df_out.head()

processing range [0,5257742]
obteneindo sub indices ...done
recorriendo datos ...
done


,Codigo,Fecha,Hora,datetime,displaced_date,minT
0,13,2010-06-11,18:45:00,2010-06-11 18:45:00,2010-06-10,3.2
1,13,2010-06-12,01:30:00,2010-06-12 01:30:00,2010-06-11,-0.1
2,13,2010-06-13,03:45:00,2010-06-13 03:45:00,2010-06-12,2.2
3,13,2010-06-14,19:45:00,2010-06-14 19:45:00,2010-06-13,3.5
4,13,2010-06-15,09:30:00,2010-06-15 09:30:00,2010-06-14,0.3


programa final

In [11]:
# 
def run(data, ranges):
    m = mp.Manager()
    result_queue = m.Queue()
    jobs = []
    for i in range(len(ranges)-1):
        ini = ranges[i]
        end = ranges[i+1]
        jobs.append(mp.Process(target=worker, args=(data, ini, end, result_queue)))
        jobs[-1].start()
    
    for p in jobs:
        p.join()
    return result_queue

In [16]:
# necesito testear esto
rqueue = run(data, ranges)
dicts = defaultdict(list)
nres = rqueue.qsize()
while nres > 0:
    dicti = rqueue.get()
    nres -= 1
    for k, v in dicti.items():
        dicts[k].extend(v)

processing range [0,26678]
processing range [26678,77475]
obteneindo sub indices ...done
recorriendo datos ...
processing range [77475,161990]
obteneindo sub indices ...done
recorriendo datos ...
done
processing range [161990,242822]
obteneindo sub indices ...done
processing range [242822,366412]
done
recorriendo datos ...
obteneindo sub indices ...done
processing range [366412,526204]
recorriendo datos ...
processing range [526204,749696]
obteneindo sub indices ...done
recorriendo datos ...
processing range [749696,1010546]
done
processing range [1010546,1286809]
obteneindo sub indices ...done
recorriendo datos ...
processing range [1286809,1560976]
done
obteneindo sub indices ...done
recorriendo datos ...
processing range [1560976,1836371]
obteneindo sub indices ...done
recorriendo datos ...
processing range [2080444,2352802]
done
obteneindo sub indices ...done
processing range [1836371,2080444]
recorriendo datos ...
processing range [2352802,2641894]
obteneindo sub indices ...done
d

In [17]:
pd.DataFrame(dicts)

,Codigo,Fecha,Hora,datetime,displaced_date,minT
0,13,2010-06-11,18:45:00,2010-06-11 18:45:00,2010-06-10,3.2
1,13,2010-06-12,01:30:00,2010-06-12 01:30:00,2010-06-11,-0.1
2,13,2010-06-13,03:45:00,2010-06-13 03:45:00,2010-06-12,2.2
3,13,2010-06-14,19:45:00,2010-06-14 19:45:00,2010-06-13,3.5
4,13,2010-06-15,09:30:00,2010-06-15 09:30:00,2010-06-14,0.3
...,...,...,...,...,...,...
56676,450050,2010-06-25,05:00:00,2010-06-25 05:00:00,2010-06-24,5.8
56677,450050,2010-06-26,07:30:00,2010-06-26 07:30:00,2010-06-25,4.4
56678,450050,2010-06-27,01:45:00,2010-06-27 01:45:00,2010-06-26,8.1
56679,450050,2010-06-27,20:30:00,2010-06-27 20:30:00,2010-06-27,8.3


## PROBLEMA:

existen dias (desplazados) que no presentan mediciones durante la noche. Estos dias no nos sirven dado que no tenemos como asignarles un label (temperatura minima y hora en que ocurre) por lo que deben ser identificados y descartados 

In [103]:
def has_night_data(data, ini, end):
    # recive los datos y un intervalo de indices que corresponde a un dia (desplazado) de medicion
    day = str(data[ini][10].date())
    start_datetime = datetime.strptime(day + " 19:00:00", '%Y-%m-%d %H:%M:%S')
    end_datetime = start_datetime + timedelta(hours=14)
#     print(start_datetime, '-', end_datetime)
    c = 0
    for i in range(ini, end):
#         print(data[i][10])
        if start_datetime.date() == data[i][10].date() and end_datetime.time() >= data[i][10].time():
#             print("add1")
            c += 1
        elif start_datetime.date() != data[i][10].date() and start_datetime.time() <= data[i][10].time():
#             print("add2")
            c += 1
    # usaremos como criterio una medicion de a lo menos 8 horas durante la noche, es decir
#     print(c)
    return c >= 8 * 4

def check_days_worker(data, ini, end, out_q):
    # toma los datos completos y recive un subintervalo dado por codigos de estaciones
    # en dicho sub intervalo, identifica los dias (desplazados) que no presentan las 
    # suficientes mediciones durante la noche
    sub_idxs = [ini]
    last_code = data[ini][0]
    for i in range(ini, end):
        if last_code != data[i][0]:
            last_code = data[i][0]
            sub_idxs.append(i)
    sub_idxs.append(end)
    
    results = []
    for i in range(len(sub_idxs) - 1):
        subini = sub_idxs[i]
        subend = sub_idxs[i+1]
        last_day = data[subini][11]
        subsubini = subini
        for j in range(subini, subend):
            if last_day != data[j][11]:
                last_day = data[j][11]
                # revisamos la noche
                if not has_night_data(data, subsubini, j):
                    results.append(subsubini)
                subsubini = j
    out_q.put(results)

def run_check(data, ranges):
    m = mp.Manager()
    result_queue = m.Queue()
    jobs = []
    for i in range(len(ranges)-1):
        ini = ranges[i]
        end = ranges[i+1]
        jobs.append(mp.Process(target=check_days_worker, args=(data, ini, end, result_queue)))
        jobs[-1].start()
    
    for p in jobs:
        p.join()
    return result_queue

# m = mp.Manager()
# result_queue = m.Queue()
# ini = ranges[1]
# end = ranges[2]
# check_days_worker(data, ini, end, result_queue)
# ds = result_queue.get()
# print(len(ds), end-ini)

rqueue = run_check(data, ranges)
rlen = rqueue.qsize()
idxs = []
while rlen > 0:
    idxs.extend(rqueue.get())
    rlen -= 1
print(len(idxs), data.shape[0])

1798 5257742


In [104]:
df.iloc[idxs]

,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica,datetime,displaced_date
0,13,2010-06-11,12:15:00,6.2,70.2,179.9,0.0,0.0,125.5,1008.0,2010-06-11 12:15:00,2010-06-10
1867,19,2010-06-14,00:15:00,10.2,78.6,289.3,2.6,0.0,0.0,1002.0,2010-06-14 00:15:00,2010-06-13
13618,24,2010-06-18,00:00:00,8.6,97.7,154.0,0.0,0.0,0.0,997.0,2010-06-18 00:00:00,2010-06-17
13966,24,2010-06-21,20:00:00,10.3,97.3,100.0,0.1,0.2,0.0,1009.0,2010-06-21 20:00:00,2010-06-21
13979,24,2010-06-24,09:15:00,7.6,98.4,260.6,0.4,0.0,191.8,1013.0,2010-06-24 09:15:00,2010-06-23
...,...,...,...,...,...,...,...,...,...,...,...,...
5250559,450050,2010-04-04,00:00:00,2.1,96.5,20.0,0.0,0.0,1.0,1026.1,2010-04-04 00:00:00,2010-04-03
5250618,450050,2010-04-05,00:00:00,2.4,97.4,152.0,0.0,0.0,1.0,1021.1,2010-04-05 00:00:00,2010-04-04
5252916,450050,2010-05-02,00:00:00,11.1,83.5,341.0,1.5,0.0,1.0,1009.4,2010-05-02 00:00:00,2010-05-01
5256837,450050,2010-06-18,00:00:00,-0.3,80.5,49.0,1.9,0.0,1.0,1018.6,2010-06-18 00:00:00,2010-06-17


In [105]:
df[(df["Codigo"] == 24) & 
   (datetime(year=2010, month=6, day=18, hour=19)<= df["datetime"]) & 
   (df["datetime"] <= datetime(year=2010, month=6, day=19, hour=9))]


,Codigo,Fecha,Hora,Temperatura,Humedad,Direccion de Viento,Velocidad de Viento,Precipitacion,Radiacion Solar,Presion Atmosferica,datetime,displaced_date
13658,24,2010-06-18,19:00:00,8.8,96.9,271.6,0.0,0.0,0.0,1004.0,2010-06-18 19:00:00,2010-06-17
13659,24,2010-06-18,19:15:00,8.8,97.3,243.9,0.0,0.0,0.0,1005.0,2010-06-18 19:15:00,2010-06-17
13660,24,2010-06-18,19:30:00,8.8,97.1,240.6,0.0,0.0,0.0,1005.0,2010-06-18 19:30:00,2010-06-17
13661,24,2010-06-18,19:45:00,8.9,97.3,294.4,0.0,0.0,0.0,1005.0,2010-06-18 19:45:00,2010-06-17
13678,24,2010-06-19,00:00:00,8.3,97.5,202.6,0.0,0.1,0.0,1007.0,2010-06-19 00:00:00,2010-06-18
13679,24,2010-06-19,00:15:00,8.2,97.8,202.5,0.0,0.0,0.0,1007.0,2010-06-19 00:15:00,2010-06-18
13680,24,2010-06-19,00:30:00,8.2,97.8,202.5,0.0,0.0,0.0,1007.0,2010-06-19 00:30:00,2010-06-18
13681,24,2010-06-19,00:45:00,8.1,97.8,234.2,0.0,0.0,0.0,1008.0,2010-06-19 00:45:00,2010-06-18
13682,24,2010-06-19,01:00:00,8.0,97.9,236.0,0.0,0.0,0.0,1008.0,2010-06-19 01:00:00,2010-06-18
13683,24,2010-06-19,01:15:00,7.9,97.9,205.4,0.0,0.0,0.0,1008.0,2010-06-19 01:15:00,2010-06-18


hay muchos dias donde la temperatura minima es durante el dia. Nosotros no buscamos predecir la temperatura a toda hora, por lo que tomaremos solamente las horas entre 20 hrs y 10 hrs

In [65]:
def select_night(data, ini, end):
    day = str(data[ini][10].date())
    start_datetime = datetime.strptime(day + " 20:00:00", '%Y-%m-%d %H:%M:%S')
    end_datetime = start_datetime + timedelta(hours=14)
    print(start_datetime, "-", end_datetime)
    i = ini
    j = end
    while i < end:
        print(data[i][10])
        if start_datetime <= data[i][10]:
            print(data[i][10])
            break
        i += 1
    while j > i:
        if end_datetime <= data[j][10]:
            print(data[j][10])
            break
        j -= 1
    print(i, j)
    return i, j

def worker2(data, ini, end, out_q):
    if True:
        results = defaultdict(list)
        print("processing range [{},{}]".format(ini, end))
        # primero obtenemos los sub indices de cada estacion (codigo)
        print("obteneindo sub indices ...", end="")
        sub_idxs = [ini]
        last_code = data[ini][0]
        for i in range(ini, end):
            if last_code != data[i][0]:
                last_code = data[i][0]
                sub_idxs.append(i)
        sub_idxs.append(end)
        print("done")
        
        # luego, recorremos estos rangos, identificamos el rango donde se encuentre las mediciones
        # para 1 solo dia y encontramos el idx donde ocurre la temperatura minima
        timewindow = timedelta(hours = 14)
        print("recorriendo datos ...")
        for i in range(len(sub_idxs) - 1):
            subini = sub_idxs[i]
            subend = sub_idxs[i+1]
            last_day = data[subini][11]
            subsubini = subini
            for j in range(subini, subend):
                dj = data[j]
                if last_day != data[j][11]:
                    last_day = data[j][11]
                    # obtenemos indices de la noche
                    night_ini, night_end = select_night(data, subsubini, j)
                    # obtenemos el arreglo de temperaturas
                    temps = data[night_ini:night_end,3]
                    idx_min = np.argmin(temps)
                    # y pasamos todo al diccionaro de salida
                    for k, name in zip([0, 1, 2, 10, 11, 3], 
                                   ["Codigo", "Fecha", "Hora", "datetime", "displaced_date", "minT"]):
                        results[name].append(data[subsubini + idx_min][k])
                    subsubini = j
        print("done")
                    
        out_q.put(results)
#     except:
#         print("worker failed")
#     finally:
#         print("done")

def run2(data, ranges):
    m = mp.Manager()
    result_queue = m.Queue()
    jobs = []
    for i in range(len(ranges)-1):
        ini = ranges[i]
        end = ranges[i+1]
        jobs.append(mp.Process(target=worker2, args=(data, ini, end, result_queue)))
        jobs[-1].start()
    
    for p in jobs:
        p.join()
    return result_queue

In [66]:
# probemos el worker2 en un pequeño rango de 1 codigo
idxs = df[df["Codigo"] == 450050].index
ini = idxs[0]
end = idxs[-1]
print(ini, end, end-ini)
m = mp.Manager()
result_queue = m.Queue()
worker2(data, ini, end, result_queue)
x = result_queue.get()
pd.DataFrame(x).head()

5225570 5257741 32171
processing range [5225570,5257741]
obteneindo sub indices ...done
recorriendo datos ...
2009-07-01 20:00:00 - 2009-07-02 10:00:00
2009-07-01 00:00:00
2009-07-01 00:15:00
2009-07-01 00:30:00
2009-07-01 00:45:00
2009-07-01 01:00:00
2009-07-01 01:15:00
2009-07-01 01:30:00
2009-07-01 01:45:00
2009-07-01 02:00:00
2009-07-01 02:15:00
2009-07-01 02:30:00
2009-07-01 02:45:00
2009-07-01 03:00:00
2009-07-01 03:15:00
2009-07-01 03:30:00
2009-07-01 03:45:00
2009-07-01 04:00:00
2009-07-01 04:15:00
2009-07-01 04:30:00
2009-07-01 04:45:00
2009-07-01 05:00:00
2009-07-01 05:15:00
2009-07-01 05:30:00
2009-07-01 05:45:00
2009-07-01 06:00:00
2009-07-01 06:15:00
2009-07-01 06:30:00
2009-07-01 06:45:00
2009-07-01 07:00:00
2009-07-01 07:15:00
2009-07-01 07:30:00
2009-07-01 07:45:00
2009-07-01 08:00:00
2009-07-01 08:15:00
2009-07-01 08:30:00
2009-07-01 08:45:00
2009-07-01 09:00:00
2009-07-01 09:15:00
2009-07-01 09:30:00
2009-07-01 09:45:00
2009-07-01 10:00:00
2009-07-01 10:15:00
2009-07-

ValueError: attempt to get argmin of an empty sequence

In [39]:
%debug

> <ipython-input-37-b9bf8d56d8f3>(62)worker2()
     60                     for k, name in zip([0, 1, 2, 10, 11, 3], 
     61                                    ["Codigo", "Fecha", "Hora", "datetime", "displaced_date", "minT"]):
---> 62                         results[name].append(data[subsubini + idx_min][k])
     63                     subsubini = j
     64         print("done")

ipdb> subsubini
5225570
ipdb> data.shape
(5257742, 12)
ipdb> idx_min
5222233
ipdb> night_ini
79
ipdb> night_end
5225650
ipdb> subsubini
5225570
ipdb> j
*** The 'jump' command requires a line number
ipdb> print(j)
5225650
ipdb> exit


In [33]:
# necesito testear esto
rqueue = run2(data, ranges)
dicts = defaultdict(list)
nres = rqueue.qsize()
while nres > 0:
    dicti = rqueue.get()
    nres -= 1
    for k, v in dicti.items():
        dicts[k].extend(v)

processing range [0,26678]
processing range [26678,77475]
obteneindo sub indices ...done
recorriendo datos ...
obteneindo sub indices ...done
processing range [77475,161990]
recorriendo datos ...
processing range [161990,242822]
obteneindo sub indices ...done
recorriendo datos ...
processing range [242822,366412]
done
obteneindo sub indices ...done
recorriendo datos ...
processing range [366412,526204]
obteneindo sub indices ...done
recorriendo datos ...
processing range [526204,749696]
obteneindo sub indices ...done
processing range [749696,1010546]
recorriendo datos ...
obteneindo sub indices ...done
processing range [1010546,1286809]
recorriendo datos ...
processing range [1286809,1560976]
obteneindo sub indices ...done
recorriendo datos ...
processing range [1560976,1836371]
obteneindo sub indices ...done
processing range [1836371,2080444]
recorriendo datos ...
processing range [2080444,2352802]
processing range [2352802,2641894]
obteneindo sub indices ...done
obteneindo sub indice

Process Process-136:


obteneindo sub indices ...done
processing range [4610683,4707174]


Traceback (most recent call last):


processing range [4707174,4913502]
obteneindo sub indices ...done
recorriendo datos ...


...

done
done


Process Process-132:
Process Process-121:
Traceback (most recent call last):
Process Process-123:


KeyboardInterrupt: 

Process Process-124:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "<ipython-input-29-c219bf9bc16a>", line 56, in worker2
    idx_min = np.argmin(temps)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "<ipython-input-29-c219bf9bc16a>", line 56, in worker2
    idx_min = np.argmin(temps)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run


  File "<__array_function__ internals>", line 6, in argmin
  File "/usr/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 1222, in argmin
    return _wrapfunc(a, 'argmin', axis=axis, out=out)
  File "/usr/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 1222, in argmin
    return _wrapfunc(a, 'argmin', axis=axis, out=out)
  File "/usr/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 1222, in argmin
    return _wrapfunc(a, 'argmin', axis=axis, out=out)
  File "<__array_function__ internals>", line 6, in argmin
  File "/usr/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 1222, in argmin
    return _wrapfunc(a, 'argmin', axis=axis, out=out)
  File "/usr/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 61, in _wrapfunc
    return bound(*args, **kwds)
  File "/usr/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 61, in _wrapfunc
    return bound(*args, **kwds)
  File "/usr/lib/python3.7/site-packages/numpy/core/fromnume

In [31]:
df_out = pd.DataFrame(dicts)

In [32]:
df_out.head()

,Codigo,Fecha,Hora,datetime,displaced_date,minT
0,13,2010-06-11,18:45:00,2010-06-11 18:45:00,2010-06-10,3.2
1,13,2010-06-12,01:30:00,2010-06-12 01:30:00,2010-06-11,-0.1
2,13,2010-06-13,03:45:00,2010-06-13 03:45:00,2010-06-12,2.2
3,13,2010-06-14,19:45:00,2010-06-14 19:45:00,2010-06-13,3.5
4,13,2010-06-15,09:30:00,2010-06-15 09:30:00,2010-06-14,0.3
